In [1]:
from elasticsearch import helpers
from elasticsearch import Elasticsearch
import json
es = Elasticsearch()

In [2]:
def bulk_load(actions_):
    """Performs the bulk load to Elasticsearch
    """
    res = None
    try:
        res = helpers.parallel_bulk(es, actions_)
    except Exception as e:
        print("There was an error")
        print(e)
    return res
def process_result(result_list):
    success, fail = 0, 0
    for i in result_list:
        if i[0]:
            success = success + 1
        else:
            fail = fail + 1
    return success, fail

In [ ]:
with open("test_data.json","r") as file:
    _actions = json.loads(file.read())

In [ ]:
result_list = list(bulk_load(_actions))
bulk_result = process_result(result_list)

In [ ]:
result_list

In [ ]:
bulk_result

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "function_score": {
            "query": {
                "match": { "message": "elasticsearch" }
            },
            "script_score" : {
                "script" : {
                  "source": "Math.log(2 + doc[\u0027likes\u0027].value)"
                }
            }
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match" : {
            "message" : "elasticsearch"
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match_phrase" : {
            "message" : "elasticsearch"
        }
    }
}'

In [6]:
def get_last_id(index, doctype):
    """Get the last ID for autoincremental ID generation"""
    _query = {
        "size": 1,
        "sort": {"created_at": "desc"},
        "query": {
            "match_all": {}
        }
    }
    _filter_path = 'hits.hits._id'
    try:
        result = es.search(
            index=index,
            doc_type=doctype,
            body=_query,
            filter_path=_filter_path)
        return int(result['hits']['hits'][0]['_id'])
    except ConnectionError as error:
        return 0
get_last_id('inventory', 'document')

GET http://localhost:9200/inventory/document/_search?filter_path=hits.hits._id [status:400 request:0.045s]


RequestError: TransportError(400, 'search_phase_execution_exception', 'No mapping found for [created_at] in order to sort on')